In [1]:
import os
import sys

# Add the repository root to the Python path
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, repo_root)

# Set the working directory to the repository root
os.chdir(repo_root)

In [1]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva.core.ermrest_model import (
    builtin_types, 
    Schema, 
    Table, 
    Column, 
    Key, 
    ForeignKey
)
from deriva_ml import DatasetSpec, ExecutionConfiguration, DerivaML, Workflow
from deriva_ml import MLVocab as vc
from deriva_ml.deriva_definitions import ColumnDefinition, BuiltinTypes
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'dev.eye-ai.org'
# host = 'www.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [3]:
cache_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
working_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
source_dataset = '2-7P5P'

EA.add_term("Workflow_Type", term_name="Feature_Creation", 
            description="Workflow for feature creation")

workflow_instance = Workflow(
    name="Create Condition_Label feature",
    workflow_type="Feature_Creation",
    url=''
    is_notebook=True
)

config = ExecutionConfiguration(
    datasets=[DatasetSpec(rid=source_dataset, version=EA.dataset_version(source_dataset), materialize=False)],
    workflow= workflow_instance, # dev'5-SG9W'
    description="Create Condition_Label for multimodal data")

# Initialize execution
execution = EA.create_execution(config)

In [ ]:
print(execution)

# Create Feature

In [6]:
# severity_cv = EA.create_vocabulary(vocab_name='Chart_Label', schema='eye-ai')
chart_label_provider = Column.define(
    'Chart_Label_Provider',
    builtin_types.text,
    nullok=True
    )
severity_feature = EA.create_feature(target_table='Subject', feature_name='Chart_Label', 
                  terms=['Image Side', 'Condition_Label', 'Severity_Label'],
                  metadata=[chart_label_provider])

# Compute Feature
## Retrived data

In [7]:
ds_bag = execution.datasets[0]
tables = ds_bag.list_tables()
# tables

In [8]:
chart_label = pd.read_csv('')

# Feature ingestion

In [ ]:
EA.find_features('Clinical_Records')

In [ ]:
feature_name = 'Chart_Label'
Feature = EA.feature_record_class('Subject', feature_name)
from IPython.display import Markdown, display
display(
    Markdown('### Feature Name'),
    [ f'Name: {c.name}, Required: {not c.nullok}' for c in Feature.feature.feature_columns]
)

In [14]:
chart_label_feature_list = [Feature(
    Execution=execution.execution_rid,
    Chart_Label_Provider=row['chart_label_provider'],
    Condition_Label=row['chart_label'],
    Severity_Label=row['severity_chart_label'],
    Feature_Name=feature_name,
    ) for index, row in chart_label.iterrows()]

In [16]:
execution.add_features(chart_label_feature_list)

In [ ]:
execution.upload_execution_outputs()